In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from dataset_test import make_dataset
from os.path import join


train_path = join('jeju_data', 'train_new.parquet')
test_path = join('jeju_data', 'test_new.parquet')

x_train, y_train, test = make_dataset(train_path, test_path)

Start time:  2022-10-22 14:39:44.581731
Train dataset success !
Test dataset success !
End time: 2022-10-22 14:40:55.710094
Play time:  0:01:11.128363


In [2]:
X = x_train.copy()
y = y_train.copy()

# x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, shuffle=True, random_state=1103)

In [3]:
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from catboost import CatBoostRegressor

In [23]:

def objective(trial):
  param = {
      "random_state":42,
      'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.05),
      'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
      "n_estimators":trial.suggest_int("n_estimators", 500, 5000),
      "max_depth":trial.suggest_int("max_depth", 4, 16),
      'random_strength' :trial.suggest_int('random_strength', 0, 100),
      "colsample_bylevel":trial.suggest_float("colsample_bylevel", 0.4, 1.0),
      "l2_leaf_reg":trial.suggest_float("l2_leaf_reg",1e-8,3e-5),
      "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
      "max_bin": trial.suggest_int("max_bin", 200, 500),
      'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
  }
  X_train, X_valid, y_train, y_valid = train_test_split(X,y,test_size=0.3)
  cat = CatBoostRegressor(**param)
  cat.fit(X_train, y_train,
          eval_set=[(X_train, y_train), (X_valid,y_valid)],
          early_stopping_rounds=50,
          verbose=100)
  cat_pred = cat.predict(X_valid)
  mae = mean_absolute_error(y_valid, cat_pred)

  return mae

sampler = TPESampler(seed=42)
study = optuna.create_study(
    study_name = 'cat_parameter_opt',
    direction = 'minimize',
    sampler = sampler,
)

study.optimize(objective, n_trials=30)
print("Best Score:",study.best_value)
print("Best trial",study.best_trial.params)

[I 2022-10-23 13:41:26,058] A new study created in memory with name: cat_parameter_opt
C:\Users\jkpark\AppData\Local\Temp\ipykernel_14420\354051608.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.05),
C:\Users\jkpark\AppData\Local\Temp\ipykernel_14420\354051608.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),


0:	learn: 15.7712344	test: 15.7712344	test1: 15.7743621	best: 15.7743621 (0)	total: 289ms	remaining: 18m 15s
100:	learn: 8.4444179	test: 8.4444179	test1: 8.4399513	best: 8.4399513 (100)	total: 27.2s	remaining: 16m 34s
200:	learn: 7.3270225	test: 7.3270225	test1: 7.3235087	best: 7.3235087 (200)	total: 53.2s	remaining: 15m 50s
300:	learn: 6.9431849	test: 6.9431849	test1: 6.9400495	best: 6.9400495 (300)	total: 1m 18s	remaining: 15m 13s
400:	learn: 6.6838840	test: 6.6838840	test1: 6.6812799	best: 6.6812799 (400)	total: 1m 44s	remaining: 14m 40s
500:	learn: 6.5009679	test: 6.5009679	test1: 6.4994151	best: 6.4994151 (500)	total: 2m 8s	remaining: 14m 6s
600:	learn: 6.3534288	test: 6.3534288	test1: 6.3531122	best: 6.3531122 (600)	total: 2m 34s	remaining: 13m 38s
700:	learn: 6.2323504	test: 6.2323504	test1: 6.2326925	best: 6.2326925 (700)	total: 2m 59s	remaining: 13m 11s
800:	learn: 6.1205720	test: 6.1205720	test1: 6.1217662	best: 6.1217662 (800)	total: 3m 25s	remaining: 12m 46s
900:	learn: 5.9

[I 2022-10-23 13:57:32,359] Trial 0 finished with value: 3.376312378072731 and parameters: {'learning_rate': 0.018272261776066247, 'bagging_temperature': 63.512210106407046, 'n_estimators': 3794, 'max_depth': 11, 'random_strength': 15, 'colsample_bylevel': 0.49359671220172163, 'l2_leaf_reg': 1.7519275289243016e-06, 'min_child_samples': 88, 'max_bin': 380, 'od_type': 'IncToDec'}. Best is trial 0 with value: 3.376312378072731.
C:\Users\jkpark\AppData\Local\Temp\ipykernel_14420\354051608.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.05),
C:\Users\jkpark\AppData\Local\Temp\ipykernel_14420\354051608.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/re

0:	learn: 15.5672418	test: 15.5672418	test1: 15.5758501	best: 15.5758501 (0)	total: 187ms	remaining: 4m 32s
100:	learn: 8.5149793	test: 8.5149793	test1: 8.5057943	best: 8.5057943 (100)	total: 16.9s	remaining: 3m 45s
200:	learn: 7.7970269	test: 7.7970269	test1: 7.7872818	best: 7.7872818 (200)	total: 33.2s	remaining: 3m 26s
300:	learn: 7.3836334	test: 7.3836334	test1: 7.3740719	best: 7.3740719 (300)	total: 49.5s	remaining: 3m 9s
400:	learn: 6.9966669	test: 6.9966669	test1: 6.9866298	best: 6.9866298 (400)	total: 1m 5s	remaining: 2m 52s
500:	learn: 6.6969027	test: 6.6969027	test1: 6.6879304	best: 6.6879304 (500)	total: 1m 21s	remaining: 2m 35s
600:	learn: 6.4836264	test: 6.4836264	test1: 6.4746684	best: 6.4746684 (600)	total: 1m 38s	remaining: 2m 19s
700:	learn: 6.3204340	test: 6.3204340	test1: 6.3120120	best: 6.3120120 (700)	total: 1m 54s	remaining: 2m 2s
800:	learn: 6.1728511	test: 6.1728511	test1: 6.1641541	best: 6.1641541 (800)	total: 2m 10s	remaining: 1m 46s
900:	learn: 6.0451528	test

[I 2022-10-23 14:01:32,417] Trial 1 finished with value: 4.136473098245934 and parameters: {'learning_rate': 0.04763628595029446, 'bagging_temperature': 21.368329072358772, 'n_estimators': 1455, 'max_depth': 6, 'random_strength': 18, 'colsample_bylevel': 0.5825453457757226, 'l2_leaf_reg': 1.5747445384650815e-05, 'min_child_samples': 46, 'max_bin': 287, 'od_type': 'IncToDec'}. Best is trial 0 with value: 3.376312378072731.
C:\Users\jkpark\AppData\Local\Temp\ipykernel_14420\354051608.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.05),
C:\Users\jkpark\AppData\Local\Temp\ipykernel_14420\354051608.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/relea

...

0:	learn: 15.5079903	test: 15.5079903	test1: 15.5126740	best: 15.5126740 (0)	total: 532ms	remaining: 33m 6s
100:	learn: 7.0358108	test: 7.0358108	test1: 7.0427731	best: 7.0427731 (100)	total: 1m 35s	remaining: 57m 25s
200:	learn: 6.5111126	test: 6.5111126	test1: 6.5222838	best: 6.5222838 (200)	total: 2m 48s	remaining: 49m 21s
300:	learn: 6.2297736	test: 6.2297736	test1: 6.2464050	best: 6.2464050 (300)	total: 4m 5s	remaining: 46m 37s
400:	learn: 5.6413859	test: 5.6413859	test1: 5.6612952	best: 5.6612952 (400)	total: 5m 53s	remaining: 49m 1s
500:	learn: 5.2292961	test: 5.2292961	test1: 5.2516287	best: 5.2516287 (500)	total: 7m 52s	remaining: 50m 54s
600:	learn: 4.9812855	test: 4.9812855	test1: 5.0066424	best: 5.0066424 (600)	total: 9m 59s	remaining: 52m 8s
700:	learn: 4.8283644	test: 4.8283644	test1: 4.8609391	best: 4.8609391 (700)	total: 12m 8s	remaining: 52m 33s
800:	learn: 4.7206827	test: 4.7206827	test1: 4.7604159	best: 4.7604159 (800)	total: 14m 16s	remaining: 52m 17s
900:	learn: 4.

[I 2022-10-24 03:22:31,853] Trial 28 finished with value: 2.9859445101518953 and parameters: {'learning_rate': 0.04814728699783611, 'bagging_temperature': 40.52044595178952, 'n_estimators': 3736, 'max_depth': 16, 'random_strength': 59, 'colsample_bylevel': 0.4861875826015146, 'l2_leaf_reg': 2.309535224342252e-05, 'min_child_samples': 58, 'max_bin': 424, 'od_type': 'Iter'}. Best is trial 28 with value: 2.9859445101518953.
C:\Users\jkpark\AppData\Local\Temp\ipykernel_14420\354051608.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.05),
C:\Users\jkpark\AppData\Local\Temp\ipykernel_14420\354051608.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releas

0:	learn: 15.5111995	test: 15.5111995	test1: 15.5078792	best: 15.5078792 (0)	total: 1.77s	remaining: 1h 28m
100:	learn: 7.4717051	test: 7.4717051	test1: 7.4820892	best: 7.4820892 (100)	total: 1m 9s	remaining: 33m 4s
200:	learn: 6.8224938	test: 6.8224938	test1: 6.8407885	best: 6.8407885 (200)	total: 2m 18s	remaining: 32m 4s
300:	learn: 6.4777624	test: 6.4777624	test1: 6.5023905	best: 6.5023905 (300)	total: 3m 21s	remaining: 30m 1s
400:	learn: 5.7690004	test: 5.7690004	test1: 5.7973105	best: 5.7973105 (400)	total: 5m 3s	remaining: 32m 36s
500:	learn: 5.2905021	test: 5.2905021	test1: 5.3186723	best: 5.3186723 (500)	total: 6m 52s	remaining: 34m 10s
600:	learn: 5.0260040	test: 5.0260040	test1: 5.0565062	best: 5.0565062 (600)	total: 8m 47s	remaining: 34m 57s
700:	learn: 4.8553439	test: 4.8553439	test1: 4.8928376	best: 4.8928376 (700)	total: 10m 44s	remaining: 35m 5s
800:	learn: 4.7382375	test: 4.7382375	test1: 4.7828505	best: 4.7828505 (800)	total: 12m 41s	remaining: 34m 42s
900:	learn: 4.65

[I 2022-10-24 04:22:18,934] Trial 29 finished with value: 3.0127825927583927 and parameters: {'learning_rate': 0.04875709506907699, 'bagging_temperature': 49.87581533270841, 'n_estimators': 2990, 'max_depth': 16, 'random_strength': 87, 'colsample_bylevel': 0.4492431292043895, 'l2_leaf_reg': 2.2785195712824778e-05, 'min_child_samples': 26, 'max_bin': 395, 'od_type': 'Iter'}. Best is trial 28 with value: 2.9859445101518953.


Best Score: 2.9859445101518953
Best trial {'learning_rate': 0.04814728699783611, 'bagging_temperature': 40.52044595178952, 'n_estimators': 3736, 'max_depth': 16, 'random_strength': 59, 'colsample_bylevel': 0.4861875826015146, 'l2_leaf_reg': 2.309535224342252e-05, 'min_child_samples': 58, 'max_bin': 424, 'od_type': 'Iter'}


Best Score: 2.9859445101518953  
Best trial {'learning_rate': 0.04814728699783611, 'bagging_temperature': 40.52044595178952, 'n_estimators': 3736, 'max_depth': 16, 'random_strength': 59, 'colsample_bylevel': 0.4861875826015146, 'l2_leaf_reg': 2.309535224342252e-05, 'min_child_samples': 58, 'max_bin': 424, 'od_type': 'Iter'}

In [24]:
params = study.best_trial.params
cat_model = CatBoostRegressor(**params).fit(X, y)
y_pred = cat_model.predict(test)

import pandas as pd

sample_submission = pd.read_csv('./jeju_data/sample_submission.csv')
sample_submission['target'] = y_pred
sample_submission.to_csv("./submit_cat_optuna.csv", index=False)

0:	learn: 15.5716201	total: 349ms	remaining: 21m 43s
1:	learn: 15.1267920	total: 1.13s	remaining: 35m 10s
2:	learn: 14.7153248	total: 2.46s	remaining: 51m 4s
3:	learn: 14.3778033	total: 2.72s	remaining: 42m 17s
4:	learn: 14.1375031	total: 2.95s	remaining: 36m 38s
5:	learn: 13.7320558	total: 4.26s	remaining: 44m 8s
6:	learn: 13.3522024	total: 5.72s	remaining: 50m 46s
7:	learn: 12.9751077	total: 7.34s	remaining: 57m 2s
8:	learn: 12.6400785	total: 8.84s	remaining: 1h 1m 2s
9:	learn: 12.3462009	total: 10.5s	remaining: 1h 5m 16s
10:	learn: 12.0403986	total: 12s	remaining: 1h 7m 40s
11:	learn: 11.8349714	total: 12.4s	remaining: 1h 4m
12:	learn: 11.5785669	total: 13.7s	remaining: 1h 5m 24s
13:	learn: 11.3470364	total: 15.3s	remaining: 1h 7m 44s
14:	learn: 11.1201618	total: 16.5s	remaining: 1h 8m 2s
15:	learn: 10.9087378	total: 17.9s	remaining: 1h 9m 12s
16:	learn: 10.7733038	total: 18.2s	remaining: 1h 6m 31s
17:	learn: 10.5773935	total: 19.7s	remaining: 1h 7m 46s
18:	learn: 10.3737512	total: 

In [25]:
import pandas as pd
sample_submission = pd.read_csv('./jeju_data/sample_submission.csv')

params = study.best_trial.params


from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

folds = []

for train_idx, val_idx in skf.split(X, y):
    folds.append((train_idx, val_idx))

CAT_model= {}

for f in range(5):
      print(f'===================================={f+1}============================================')
      train_idx, val_idx = folds[f]
      
      x_train, x_val, y_train, y_val = X.iloc[train_idx], X.iloc[val_idx], y.iloc[train_idx], y.iloc[val_idx]
      
      CAT = CatBoostRegressor(**params)
      CAT.fit(x_train, y_train)
      
      y_pred = CAT.predict(x_val)
      mae = mean_absolute_error(y_val, y_pred)
      print(f"{f + 1} Fold MAE = {mae}")
      CAT_model[f] = CAT
      print(f'================================================================================\n\n')
              

for fold in range(5):
    sample_submission['target'] += CAT_model[fold].predict(test)/5    
sample_submission.to_csv("./catcat.csv", index=False)

In [30]:
a = pd.read_csv('catcat.csv')
b = pd.read_csv('./model/XGB/public 3.14 xgb.csv')

In [32]:
c = (a['target']*0.5) + (b['target']*0.5)
sample_submission = pd.read_csv('./jeju_data/sample_submission.csv')
sample_submission['target'] = c
sample_submission.to_csv("./submit_ff.csv", index=False)